In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.linalg import block_diag

In [3]:
from denoise import laloux
from srinkage_linear import ledoit_wolf_lin
from srinkage_nonlinear import ledoit_wolf_nonlin
from srinkage_quadratic import ledoit_wolf_quad

# 

In [4]:
# Monte Carlo experiments are based on those described in López de Prado (2020), Mashine Learning for Asset Managers

In [5]:
def eigh_decompose(matrix):
    """
        Return the eigenvalues and eigenvectors in a sorted order 
    """
    eigenVal, eigenVec = np.linalg.eigh(matrix)
    indices = eigenVal.argsort()[::-1]
    eigenVal, eigenVec = eigenVal[indices], eigenVec[:,indices]
    eigenVal = np.diagflat(eigenVal)    
    return eigenVal, eigenVec

def form_block_matrix(nBlocks, bSize, bCorr):
    """
        Form correlation matrix out of 'nBlocks' blocks of size 'bSize' each, where off - diagonal elements within each block have a correlation of 'bCorr'.
        This covariance matrix is a stylized representation of a true (nonempirical) correlation matrix of the S&P 500, where each block is associated with an economic sector.
    """    
    block=np.ones((bSize, bSize)) * bCorr
    block[range(bSize),range(bSize)]=1
    corr=block_diag(*([block]*nBlocks))
    return corr

def form_true_matrix(nBlocks, bSize, bCorr):
    """
        Without loss of generality, the variances are drawn from a uniform distribution bounded between 5% and 20%,
        and the vector of means is drawn from a Normal distribution with mean and standard deviation equal to the standard deviation from the covariance matrix.
        This is consistent with the notion that in an efficient market all securities have the same expected Sharpe ratio. 
        Returns vector of means of lenght N and covariance matrix formed from "true" (nonempirical) correlation matrix and vector of variances
    """
    corr0 = form_block_matrix(nBlocks, bSize, bCorr) 
    corr0 = pd.DataFrame(corr0)
    cols = corr0.columns.tolist()
    np.random.shuffle(cols) 
    corr0 = corr0[cols].loc[cols].copy(deep=True)
    std0 = np.random.uniform(.05, .2, corr0.shape[0]) 
    cov0 = corr2cov(corr0, std0)
    mu0 = np.random.normal(std0, std0, cov0.shape[0]).reshape(-1,1)
    return mu0, cov0


def sim_cov_mu(mu0, cov0, nObs):
    """
        Uses the "true" (nonempirical) covariance matrix to draw a random matrix X of size TxN,
        and it derives the associated empirical covariance matrix and vector of means.
        
        nObs: sets the value of T.
        
        When shrink = True, the function performs a Ledoit – Wolf shrinkage of the empirical covariance matrix.
    """
    x = np.random.multivariate_normal(mu0.flatten(), cov0, size=nObs) 
    mu1 = x.mean(axis=0).reshape(-1,1)
    cov1 = np.cov(x, rowvar=0)
    return mu1, cov1, x

def denoise_cov(cov0, q, bwidth):
    """
        Given original covariance matrix, return denoised covariance matrix
    """
    corr0 = cov2corr(cov0)
    eigenVal0, eigenVec0 = eigh_decompose(corr0)
    eigenMax0, var0 = get_max_eigenVal(np.diag(eigenVal0), q, bwidth) 
    nFacts0 = eigenVal0.shape[0] - np.diag(eigenVal0)[::-1].searchsorted(eigenMax0)
    corr1 = denoised_corr(eigenVal0, eigenVec0, nFacts0)
    cov1 = corr2cov(corr1, np.diag(cov0)**.5)
    return cov1

def corr2cov(corr, std):
    cov = corr * np.outer(std, std)
    return cov


def cov2corr(cov):
    """
        Derive the correlation matrix from a covariance matrix
    """
    std = np.sqrt(np.diag(cov))
    corr = cov / np.outer(std, std)
    corr[corr<-1], corr[corr>1] = -1, 1
    return corr

def opt_portfolio(cov, mu=None):
    """    
        Derive the minimum variance portfolio
    """    
    inv = np.linalg.inv(cov)
    ones = np.ones(shape=(inv.shape[0],1))
    if mu is None:
        mu=ones
    w = np.dot(inv,mu) 
    w /= np.dot(ones.T,w)
    return w

### Minimum variance portfolio

In [6]:
nBlocks, bSize, bCorr = 10, 50, .5
np.random.seed(0)

mu0, cov0 = form_true_matrix(nBlocks, bSize, bCorr)

In [7]:
nObs, nTrials, bwidth, minVarPortf = 1000, 100, .01, True

w1 = pd.DataFrame(columns=range(cov0.shape[0]), index=range(nTrials), dtype=float)
w1_lin = w1.copy()
w1_nonlin = w1.copy()
w1_quad = w1.copy()
w1_l = w1.copy()
# fix random seed
np.random.seed(0)

for i in range(nTrials):
    mu1, cov1, x = sim_cov_mu(mu0, cov0, nObs)
    if minVarPortf:
        mu1=None 
    cov1_lin = ledoit_wolf_lin(x, d=False)
    cov1_nonlin = ledoit_wolf_nonlin(cov1, nObs)
    cov1_quad = ledoit_wolf_quad(cov1, nObs)
    cov1_l = laloux(cov1, nObs)
    
    w1.loc[i] = opt_portfolio(cov1, mu1).flatten()     
    w1_lin.loc[i] = opt_portfolio(cov1_lin, mu1).flatten()    
    w1_nonlin.loc[i] = opt_portfolio(cov1_nonlin, mu1).flatten()
    w1_quad.loc[i] = opt_portfolio(cov1_quad, mu1).flatten()
    w1_l.loc[i] = opt_portfolio(cov1_l, mu1).flatten()

In [8]:
w0 = opt_portfolio(cov0, None if minVarPortf else mu0)
w0 = np.repeat(w0.T, w1.shape[0], axis=0) 

rmsd = np.mean((w1 - w0).values.flatten() ** 2) ** .5
rmsd_lin = np.mean((w1_lin - w0).values.flatten() ** 2) ** .5
rmsd_nonlin = np.mean((w1_nonlin - w0).values.flatten() ** 2) ** .5
rmsd_quad = np.mean((w1_quad - w0).values.flatten() ** 2) ** .5
rmsd_l = np.mean((w1_l - w0).values.flatten() ** 2) ** .5 

print(rmsd, rmsd_lin, rmsd_nonlin, rmsd_quad, rmsd_l)

0.004962279513262569 0.0038560404543460033 0.0030993898354189143 0.0031016930644155384 0.0011794654557021292


### Maximum Sharpe ratio portfolio

In [9]:
nObs, nTrials, bwidth, minVarPortf = 1000, 100, .01, False

w1 = pd.DataFrame(columns=range(cov0.shape[0]), index=range(nTrials), dtype=float)
w1_lin = w1.copy()
w1_nonlin = w1.copy()
w1_quad = w1.copy()
w1_l = w1.copy()
# fix random seed
np.random.seed(0)

for i in range(nTrials):
    mu1, cov1, x = sim_cov_mu(mu0, cov0, nObs)
    if minVarPortf:
        mu1=None 
    cov1_lin = ledoit_wolf_lin(x, d=False)
    cov1_nonlin = ledoit_wolf_nonlin(cov1, nObs)
    cov1_quad = ledoit_wolf_quad(cov1, nObs)
    cov1_l = laloux(cov1, nObs)
    
    w1.loc[i] = opt_portfolio(cov1, mu1).flatten()     
    w1_lin.loc[i] = opt_portfolio(cov1_lin, mu1).flatten()    
    w1_nonlin.loc[i] = opt_portfolio(cov1_nonlin, mu1).flatten()
    w1_quad.loc[i] = opt_portfolio(cov1_quad, mu1).flatten()
    w1_l.loc[i] = opt_portfolio(cov1_l, mu1).flatten()

In [10]:
w0 = opt_portfolio(cov0, None if minVarPortf else mu0)
w0 = np.repeat(w0.T, w1.shape[0], axis=0) 

rmsd = np.mean((w1 - w0).values.flatten() ** 2) ** .5
rmsd_lin = np.mean((w1_lin - w0).values.flatten() ** 2) ** .5
rmsd_nonlin = np.mean((w1_nonlin - w0).values.flatten() ** 2) ** .5
rmsd_quad = np.mean((w1_quad - w0).values.flatten() ** 2) ** .5
rmsd_l = np.mean((w1_l - w0).values.flatten() ** 2) ** .5 

print(rmsd, rmsd_lin, rmsd_nonlin, rmsd_quad, rmsd_l)

0.20341423469941783 0.5371714505927292 0.033673430755731405 0.0337988891890699 0.015083443909607072


In [11]:
# In both portfolios linear shrinkage provides the smallest reduction in RMSE, non-linear and quadratic shrinkage are competing.
# The largest reduction in RMSE is obtained by denoising (Laloux (2000))